# Ungraded Lab: Building a Custom Dense Layer

In this lab, we'll walk through how to create a custom layer that inherits the [Layer](https://keras.io/api/layers/base_layer/#layer-class) class. Unlike simple Lambda layers you did previously, the custom layer here will contain weights that can be updated during training.

## Imports

In [3]:
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import numpy as np

## Custom Layer with weights

To make custom layer that is trainable, we need to define a class that inherits the [Layer](https://keras.io/api/layers/base_layer/#layer-class) base class from Keras. The Python syntax is shown below in the class declaration. This class requires three functions: `__init__()`, `build()` and `call()`. These ensure that our custom layer has a *state* and *computation* that can be accessed during training or inference.

In [4]:
# inherit from this base class
from tensorflow.keras.layers import Layer

class SimpleDense(Layer):

    def __init__(self, units=32):
        '''Initializes the instance attributes'''
        super(SimpleDense, self).__init__()
        self.units = units

    def build(self, input_shape):
        '''Create the state of the layer (weights)'''
        # initialize the weights
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name="kernel",
            initial_value=w_init(shape=(input_shape[-1], self.units),
                                 dtype='float32'),
            trainable=True)

        # initialize the biases
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name="bias",
            initial_value=b_init(shape=(self.units,), dtype='float32'),
            trainable=True)

    def call(self, inputs):
        '''Defines the computation from inputs to outputs'''
        return tf.matmul(inputs, self.w) + self.b

Now we can use our custom layer like below:

In [7]:
tf.ones?

In [21]:
# declare an instance of the class
my_dense = SimpleDense(units=2)

# define an input and feed into the layer
x = tf.ones((1, 2))
y = my_dense(x)

# parameters of the base Layer class like `variables` can be used
print(my_dense.variables)

[<tf.Variable 'simple_dense_10/kernel:0' shape=(2, 2) dtype=float32, numpy=
array([[-0.00811401,  0.00676195],
       [ 0.10125976, -0.02970896]], dtype=float32)>, <tf.Variable 'simple_dense_10/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>]


Let's then try using it in simple network:

In [23]:
x

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[1., 1.]], dtype=float32)>

In [11]:
np.concatenate?

In [37]:
# define the dataset
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
xs2 = np.array([-2.0, -2.0, -2.0, -2.0, -2.0, -2.0], dtype=float)
x = np.concatenate( (xs, xs2), axis=0).reshape((-1,2)) 
x

array([[-1.,  0.],
       [ 1.,  2.],
       [ 3.,  4.],
       [-2., -2.],
       [-2., -2.],
       [-2., -2.]])

In [38]:
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)
ys = np.expand_dims(ys, axis=1)
ys

array([[-3.],
       [-1.],
       [ 1.],
       [ 3.],
       [ 5.],
       [ 7.]])

In [42]:
# define the dataset
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
xs2 = np.array([-2.0, -2.0, -2.0, -2.0, -2.0, -2.0], dtype=float)
x = np.concatenate( (xs, xs2), axis=0).reshape((-1,2)) 

ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)
ys = np.expand_dims(ys, axis=1)


# use the Sequential API to build a model with our custom layer
my_layer = SimpleDense(units=1)
model = tf.keras.Sequential([my_layer])

# configure and train the model
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit( x, ys, epochs=500,verbose=1)

Train on 6 samples
Epoch 1/500
6/6 [==============================] - 0s 23ms/sample - loss: 16.2816
Epoch 2/500
6/6 [==============================] - 0s 182us/sample - loss: 14.4972
Epoch 3/500
6/6 [==============================] - 0s 151us/sample - loss: 13.2626
Epoch 4/500
6/6 [==============================] - 0s 146us/sample - loss: 12.3991
Epoch 5/500
6/6 [==============================] - 0s 143us/sample - loss: 11.7867
Epoch 6/500
6/6 [==============================] - 0s 140us/sample - loss: 11.3443
Epoch 7/500
6/6 [==============================] - 0s 160us/sample - loss: 11.0176
Epoch 8/500
6/6 [==============================] - 0s 160us/sample - loss: 10.7699
Epoch 9/500
6/6 [==============================] - 0s 145us/sample - loss: 10.5764
Epoch 10/500
6/6 [==============================] - 0s 149us/sample - loss: 10.4205
Epoch 11/500
6/6 [==============================] - 0s 141us/sample - loss: 10.2908
Epoch 12/500
6/6 [==============================] - 0s 153us/sample

In [43]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_dense_17 (SimpleDense multiple                  3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [46]:
# perform inference
print(model.predict([[10.0, 4.0]]))



[[14.204527]]


In [47]:
# see the updated state of the variables
print(my_layer.variables)

[<tf.Variable 'sequential_11/simple_dense_17/kernel:0' shape=(2, 1) dtype=float32, numpy=
array([[ 2.1627598],
       [-2.593192 ]], dtype=float32)>, <tf.Variable 'sequential_11/simple_dense_17/bias:0' shape=(1,) dtype=float32, numpy=array([2.949698], dtype=float32)>]


In [49]:
10*2.1627598 + 4*(-2.593192) + 2.949698

14.204527999999998